In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

In [2]:
#checkins = pd.read_csv('checkins.dat', header=0, skipinitialspace = True, names=['lat', 'lng'], usecols = [3,4], engine='python', sep = '|', skipfooter=1)
# for the big file- from terminal

In [3]:
data = pd.read_csv('./chekins1.csv')

In [4]:
data.head()

,id,user_id,venue_id,latitude,longitude
0,984222,15824,5222,38.895112,-77.036366
1,984234,44652,5222,33.800745,-84.410520
2,984291,105054,5222,45.523452,-122.676207
3,984318,2146539,5222,40.764462,-111.904565
4,984232,93870,380645,33.448377,-112.074037


Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас
С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [103]:
data_chunk=data[['latitude','longitude']][:100000] # 100000 rows for faster calculation

In [104]:
data_chunk.shape

(100000, 2)

In [105]:
data_chunk.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [106]:
X=data_chunk.values

In [107]:
X

array([[  38.8951118,  -77.0363658],
       [  33.800745 ,  -84.41052  ],
       [  45.5234515, -122.6762071],
       ..., 
       [  29.7628844,  -95.3830615],
       [  32.802955 ,  -96.769923 ],
       [  37.7749295, -122.4194155]])

In [108]:
from sklearn.cluster import MeanShift

In [109]:
clf_ms = MeanShift(bandwidth=0.1, min_bin_freq=15, n_jobs=3) #bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [110]:
clf_ms.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=15,
     n_jobs=3, seeds=None)

In [111]:
cluster_centers = clf_ms.cluster_centers_

20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [112]:
office_locations = {
    "Los Angeles":(33.751277, -118.188740),
    "Miami":(25.867736, -80.324116),
    "London":(51.503016, -0.075479),
    "Amsterdam":(52.378894, 4.885084),
    "Beijing":(39.366487, 117.036146),
    "Sydney":(-33.868457, 151.205134)
    }

In [113]:
for key in office_locations.keys():
    print(key,)

Beijing
London
Los Angeles
Miami
Amsterdam
Sydney


In [125]:
def dist_2d(first, second):
    return np.sqrt((first[0] - second[0]) ** 2 + (first[1] - second[1]) ** 2)

def dist_to_offices(x):
    distances = [(x, dist_2d(x, y)) for key, y in office_locations.items()]
    print(distances)
    return min(distances, key = lambda t: t[1])

In [124]:
print(distances)

NameError: name 'distances' is not defined

In [126]:
 for y in office_locations.items():
        print( y)

('Beijing', (39.366487, 117.036146))
('London', (51.503016, -0.075479))
('Los Angeles', (33.751277, -118.18874))
('Miami', (25.867736, -80.324116))
('Amsterdam', (52.378894, 4.885084))
('Sydney', (-33.868457, 151.205134))


In [127]:
closest_offices = map(dist_to_offices, cluster_centers)

In [117]:
sorted(closest_offices, key= lambda x: x[1])[:20]

[(array([ -33.86063043,  151.20477593]), 0.007834758163107856),
 (array([ 52.37296399,   4.89231722]), 0.0093533161859922255),
 (array([ 25.84567226, -80.3188906 ]), 0.022674066158385495),
 (array([ 51.50299126,  -0.12553729]), 0.050058294822787869),
 (array([  33.80987796, -118.14892381]), 0.070847732427199731),
 (array([ 25.78581242, -80.21793804]), 0.13410903336184654),
 (array([ -34.00190615,  151.12806905]), 0.154102829806012),
 (array([ 25.70534972, -80.28342874]), 0.16740596425035326),
 (array([ 26.01009825, -80.19999059]), 0.18887596060185083),
 (array([ -33.9522629,  151.0321372]), 0.19222726571703244),
 (array([  33.88832534, -118.04892817]), 0.19577945647763628),
 (array([  33.87298601, -118.36209115]), 0.21181053682436798),
 (array([  33.97257482, -118.16837067]), 0.22223329073179071),
 (array([ 51.42676329,  -0.30373207]), 0.24065314956287862),
 (array([ 52.388501  ,   4.63376547]), 0.25150208676235281),
 (array([ 51.5741517,   0.1838708]), 0.26892862728711098),
 (array([ 

from site https://www.mapcustomizer.com we find that the closed destination ([ 52.37296399,   4.89231722]) in Amsterdam, close to Madame Tussaud museum

In [84]:
locations.values()

dict_values([[-33.8386344, 151.2071142], [-33.8386344, 151.2071142]])